# to test the delay between sensor and serial connection I will try the code below

In [ ]:
import serial
import threading
import datetime

def send_to_arduino(event_type, arduino_ser):
    """
    Send the event data to Arduino over serial.
    The Arduino will generate a digital pulse based on this input.
    """
    send_time = datetime.datetime.now()  # Record time before sending
    if event_type == "Right":
        arduino_ser.write(b'R')  # Right poke event
    elif event_type == "Left":
        arduino_ser.write(b'L')  # Left poke event
    elif event_type == "Pellet":
        arduino_ser.write(b'P')  # Pellet intake event
    elif event_type == "LeftWithPellet":
        arduino_ser.write(b'W')  # Left poke with pellet event
    elif event_type == "RightWithPellet":
        arduino_ser.write(b'Q')  # Right poke with pellet event
    print(f"Sent to Arduino: {event_type} at {send_time}")

    # Wait for Arduino response
    try:
        arduino_response = arduino_ser.readline().decode('utf-8', errors='replace').strip()
        receive_time = datetime.datetime.now()  # Record time when response is received
        print(f"Arduino responded at {receive_time}: {arduino_response}")

        # Calculate the delay
        delay = receive_time - send_time
        print(f"Delay between sending and receiving: {delay.total_seconds()} seconds")
    except UnicodeDecodeError as e:
        print(f"Error decoding Arduino response: {e}")


def read_from_port(ser, arduino_ser):
    while True:
        data = ser.readline().decode('utf-8').strip()
        data_list = data.split(",")
        data_list = data_list[1:]  # Skip the FED device timestamp

        print(f"Data from {ser.port}: {data}")

        if len(data_list) == 17:  # Adjusted to the correct length
            event_type = data_list[9]  # Assuming "Event" is at index 9
            send_to_arduino(event_type, arduino_ser)
        else:
            print(f"Warning: Data length {len(data_list)} does not match expected length.")

# Define your ports and baud rate
fed_ports = ["COM16"]  # Replace with your FED COM ports
arduino_port = "COM38"  # Replace with your Arduino COM port
baud_rate = 115200

# Setup Arduino serial connection
arduino_ser = serial.Serial(arduino_port, baud_rate)

# Start reading from each FED port in a separate thread
for port in fed_ports:
    ser = serial.Serial(port, baud_rate)
    threading.Thread(target=read_from_port, args=(ser, arduino_ser)).start()


# and flash the Arduino board with this code
# int ledPin = 10; // Pin for output pulse

# void setup() {
#   Serial.begin(115200);
#   pinMode(ledPin, OUTPUT); // Set pin 10 as an output
# }

# void loop() {
#   if (Serial.available() > 0) {
#     char eventType = Serial.read(); // Read the event type identifier

#     // Blink the LED
#     digitalWrite(ledPin, HIGH);  
#     delay(10);  // 10 ms pulse duration
#     digitalWrite(ledPin, LOW);   

#     // Send a response back to Python
#     Serial.print("Blink done for event: ");
#     Serial.println(eventType);
#   }
# }
